In [1]:
import numpy as np
import pandas as pd
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# Settings
num_agents = 100
num_weeks = 50
noise_mean = 0
noise_std = 5  # standard deviation of exogenous noise

# Memory decay weights
memory_decay_weights_map = {
    2: [0.7, 0.3],
    3: [0.5, 0.3, 0.2],
    4: [0.4, 0.3, 0.2, 0.1]
}

# Initialize agents
agents = [{
    'Agent_ID': i,
    'Strategy': random.choice(['Optimist', 'Pessimist', 'Average']),
    'Memory_Length': random.choice([2, 3, 4]),
    'Risk_Tolerance': random.randint(50, 75)
} for i in range(num_agents)]

attendance_history = []
records = []

# Simulation with exogenous noise
for week in range(1, num_weeks + 1):
    decisions = {}
    
    # Agents decide based on memory-decay forecast
    for agent in agents:
        mem_len = agent['Memory_Length']
        strat = agent['Strategy']
        rt = agent['Risk_Tolerance']
        
        # Get last mem_len weeks
        memory = attendance_history[-mem_len:] if len(attendance_history) >= mem_len else []
        weights = memory_decay_weights_map[mem_len]
        
        # Default if not enough history
        if len(memory) < mem_len:
            if strat == 'Optimist':
                decision = 'Go'
            elif strat == 'Pessimist':
                decision = 'Stay'
            else:
                decision = 'Go' if random.random() < 0.5 else 'Stay'
            A_w = None
        else:
            # Weighted attendance forecast
            A_w = sum(w * a for w, a in zip(weights, memory))
            if strat == 'Optimist':
                decision = 'Go' if A_w <= rt else 'Stay'
            elif strat == 'Pessimist':
                trend_ok = memory[-1] <= sum(memory[:-1]) / len(memory[:-1])
                decision = 'Go' if (A_w <= rt and trend_ok) else 'Stay'
            else:
                decision = 'Go' if A_w <= rt else 'Stay'
        
        decisions[agent['Agent_ID']] = decision
        records.append({
            'Week': week,
            'Agent_ID': agent['Agent_ID'],
            'Strategy': strat,
            'Memory_Length': mem_len,
            'Risk_Tolerance': rt,
            'A_weighted': A_w,
            'Decision': decision,
            'Actual_Attendance': None
        })
    
    # Base attendance from agents
    base_attendance = sum(1 for d in decisions.values() if d == 'Go')
    # Add exogenous Gaussian noise and clamp
    noise = int(np.random.normal(noise_mean, noise_std))
    actual = max(0, min(num_agents, base_attendance + noise))
    
    attendance_history.append(actual)
    
    # Fill actual attendance in the last block of records
    for rec in records[-num_agents:]:
        rec['Actual_Attendance'] = actual

# Build DataFrame
df = pd.DataFrame(records)
print(df.head())


   Week  Agent_ID  Strategy  Memory_Length  Risk_Tolerance  A_weighted  \
0     1         0   Average              2              50         NaN   
1     1         1   Average              3              57         NaN   
2     1         2  Optimist              2              73         NaN   
3     1         3  Optimist              4              73         NaN   
4     1         4   Average              2              68         NaN   

  Decision  Actual_Attendance  
0       Go                 55  
1       Go                 55  
2       Go                 55  
3       Go                 55  
4       Go                 55  


In [2]:
# Save the DataFrame to CSV
csv_path = "../data/el_farol.csv"
df.to_csv(csv_path, index=False)

In [3]:
# Reload the CSV to confirm content
df_loaded = pd.read_csv(csv_path)
df_loaded.head()

,Week,Agent_ID,Strategy,Memory_Length,Risk_Tolerance,A_weighted,Decision,Actual_Attendance
0,1,0,Average,2,50,NaN,Go,55
1,1,1,Average,3,57,NaN,Go,55
2,1,2,Optimist,2,73,NaN,Go,55
3,1,3,Optimist,4,73,NaN,Go,55
4,1,4,Average,2,68,NaN,Go,55


In [4]:
# Filter the DataFrame for rows where Agent_ID == 1
agent_1_data = df[df["Agent_ID"] == 1]
agent_1_data.head(50)

,Week,Agent_ID,Strategy,Memory_Length,Risk_Tolerance,A_weighted,Decision,Actual_Attendance
1,1,1,Average,3,57,NaN,Go,55
101,2,1,Average,3,57,NaN,Go,53
201,3,1,Average,3,57,NaN,Stay,65
301,4,1,Average,3,57,56.4,Go,56
401,5,1,Average,3,57,57.2,Stay,60
501,6,1,Average,3,57,61.3,Stay,51
601,7,1,Average,3,57,56.2,Go,71
701,8,1,Average,3,57,59.5,Stay,55
801,9,1,Average,3,57,57.8,Stay,54
901,10,1,Average,3,57,62.8,Stay,66
